In [1]:
from datasets import load_dataset_builder

dataset_builder = load_dataset_builder('fancyzhx/ag_news')
print(dataset_builder.cache_dir)
print(dataset_builder.info.features)
print(dataset_builder.info.splits)

/Users/miguelangellieramontano/Documents/ambiente/pruebas/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/miguelangellieramontano/.cache/huggingface/datasets/fancyzhx___ag_news/default/0.0.0/eb185aade064a813bc0b7f42de02595523103ca4
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}
{'train': SplitInfo(name='train', num_bytes=29832303, num_examples=120000, shard_lengths=None, dataset_name='ag_news'), 'test': SplitInfo(name='test', num_bytes=1880424, num_examples=7600, shard_lengths=None, dataset_name='ag_news')}


In [2]:
import numpy as np
import cv2
from datasets import load_dataset

dataset = load_dataset('fancyzhx/ag_news')

# Write some Text
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,10)
fontScale              = 0.5
fontColor              = (255,255,255)
thickness              = 2
lineType               = 2


In [5]:
def draw_text(text: str, img_height: int = 256, img_width: int = 128) -> np.ndarray:
    # Create a black image
    img = np.zeros((img_width,img_height,3), np.uint8)
    
    # Write some Text
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    fontScale              = 0.4
    fontColor              = (255,255,255)
    thickness              = 1
    lineType               = 2

    y = cv2.getTextSize(text, font, fontScale, thickness)[0][1]
    bottomLeftCornerOfText = (0, y)

    space_width = cv2.getTextSize(" ", font, fontScale, thickness)[0][0]

    acc_width = 0
    for word in text.split(" "):
        textsize = cv2.getTextSize(word, font, fontScale, thickness)[0]
        width = textsize[0]
        height = textsize[1]

        bottomLeftCornerOfText = (acc_width,y)

        acc_width += width + space_width
        
        if acc_width > img_height:
            y = bottomLeftCornerOfText[1] + height * 2
            bottomLeftCornerOfText = (0,y)
            acc_width = width + space_width
            
        
        cv2.putText(img,word,
                    bottomLeftCornerOfText, 
                    font, 
                    fontScale,
                    fontColor,
                    thickness,
                    lineType)
        
    return img
        

In [6]:

#Display the image
img = draw_text(dataset['train'][0]['text'])
cv2.imwrite("out.jpg", img)

True

In [7]:
import os
import pandas as pd
from torchvision.io import decode_image
from torch.utils.data import Dataset

from datasets import Array3D

# train = dataset['train'].map(lambda instance: {"image": draw_text(instance['text'])})
# test = dataset['test'].map(lambda instance: {"image": draw_text(instance['text'])})
# new_features = train.features.copy()
# new_features["image"] = Array3D(shape=(128,256,3), dtype="float32")
# train = train.cast(new_features)
# test = test.cast(new_features)

# ds_torch = train.with_format('torch')

class TextToImageDataset(Dataset):
    def __init__(self, dataset='fancyzhx/ag_news', split='train', transform=None, target_transform=None):

        dataset = load_dataset(dataset, split=split)
        new_dataset = dataset.map(lambda instance: {"image": draw_text(instance['text'])})
        new_features = new_dataset.features.copy()
        new_features["image"] = Array3D(shape=(128,256,3), dtype="float32")
        new_dataset = new_dataset.cast(new_features)

        new_dataset = new_dataset.with_format('torch')
        # fix the tensor format to torch NCHW
        self.images = new_dataset['image'].permute(0, 3, 1, 2)
        self.labels = new_dataset['label']

        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.images[0]
        label = self.labels[0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


In [8]:
train_ds = TextToImageDataset()
test_ds = TextToImageDataset(split='test')

train_ds[0]

Casting the dataset:   4%|▍         | 5000/120000 [00:24<09:31, 201.21 examples/s]


OSError: [Errno 28] No space left on device

In [19]:
torch_images = ds_torch['image']

In [20]:
torch_images.size()

torch.Size([7600, 128, 256, 3])

In [24]:
dataset['train']['label'][0]

2

In [16]:
cv2.imwrite("out.jpg", np.array(ds[0]['image']))

[ WARN:0@1.018] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


True

In [17]:
ds[0]['image'].size()

AttributeError: 'list' object has no attribute 'size'

In [ ]:
import torch
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset['train']['text'], batch_size=32, num_workers=4, pin_memory=True) 

def transform_image(instance):
    # Si la columna 'image' ya es un PIL Image (usando HFImageFeature), conviértela a tensor.
    # Si es un array de numpy, conviértelo a tensor.
    image = instance['image']
    # if isinstance(image, Image.Image):
    #     # Ejemplo: convertir PIL a tensor
    #     from torchvision import transforms
    #     transform = transforms.ToTensor()
    #     instance['image'] = transform(image)
    if isinstance(image, list):
        instance['image'] = torch.FloatTensor(image).permute(2, 0, 1) # Asegurar CHW si es RGB
    return instance

# Aplicar transformaciones si no se hicieron antes.
# Puedes aplicar transforms directamente al dataset antes de crear el DataLoader
# Esto es mejor si quieres que las transformaciones se cacheen.
ds_transformed = ds.map(transform_image)



Map:  32%|███▏      | 2406/7600 [03:19<06:57, 12.44 examples/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x107cfe0d0>>
Traceback (most recent call last):
  File "/Users/miguelangellieramontano/Documents/ambiente/pruebas/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Map:  34%|███▍      | 2568/7600 [03:32<07:10, 11.69 examples/s]

In [ ]:

# Crear el DataLoader
# num_workers > 0 es clave para la carga paralela
# pin_memory=True puede ayudar si tienes GPU, ya que los tensores se moverán más rápido a la GPU
train_dataloader = DataLoader(ds_transformed, batch_size=32, num_workers=4, pin_memory=True) # Ajusta num_workers según tus CPU cores

# Iterar sobre el DataLoader (esto simula el bucle de entrenamiento)
print("Iniciando iteración del DataLoader...")
import time
start_time = time.time()
for i, batch in enumerate(train_dataloader):
    images_batch = batch['image']
    text_batch = batch['text']
    # Aquí usarías images_batch y text_batch en tu modelo
    # print(f"Batch {i}: Image shape: {images_batch.shape}, Text type: {type(text_batch)}")
    if i == 5: # Solo para demostrar los primeros batches
        break
end_time = time.time()
print(f"Tiempo para cargar 5 batches: {end_time - start_time:.2f} segundos")
